In [1]:
import os

train_done = True
study_id = 'original'
data_id = 'pan'

size = '25p'
per = 150500

data_ = 'TransMut'
encoding_ = 'embedding'

id_ = 'OF_TData_pad14_{}_onlyEmbedding_NOPOS'.format(size)

data_dir = '/home/s202357/thesis/transmut/data/transmut_github/'
model_folder = '/home/s202357/thesis/transmut/pipeline/{}/models/'.format(study_id)

result_folder = '/home/s202357/thesis/transmut/pipeline/{}/results/'.format(study_id)

plot_name = result_folder + id_+ '_plot_'
df_name = result_folder + id_+ '_df_'

if not os.path.exists(model_folder):
    print('creating dirs')
    os.makedirs(model_folder)
    os.makedirs(result_folder)
else:
    print('dirs exist')


dirs exist


In [2]:
!gpustat

mnemosyne            Tue Jun  7 15:55:23 2022  470.103.01
[0] NVIDIA TITAN V   | 39'C,   4 % |  1383 / 12066 MB | s202357(1379M)
[1] NVIDIA TITAN V   | 45'C,   9 % |  7602 / 12066 MB | s202357(3309M) s202357(4289M)
[2] NVIDIA TITAN V   | 39'C,   0 % |  1345 / 12066 MB | s202357(1341M)
[3] NVIDIA TITAN V   | 61'C,  36 % |  3702 / 12066 MB | andab(1099M) andab(2599M)
[4] NVIDIA TITAN V   | 42'C,  14 % |  1617 / 12066 MB | ansup(1615M)
[5] NVIDIA TITAN V   | 33'C,   0 % |  3062 / 12066 MB | ansup(1617M) ansup(1441M)
[6] NVIDIA TITAN V   | 54'C,  36 % |  2603 / 12066 MB | andab(2599M)


In [3]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [4]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, accuracy_score
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
#from sklearn.metrics import classification_report
#from sklearn.utils import class_weight

import gc
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

#import framework as fr

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    use_cuda = True
    print('There are %d GPU(s) avalable.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) avalable.


In [6]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [7]:
def best_treshold(df_results):
    acc_best = -999
    best_i = -999

    for i in np.arange(0, 1, 0.1):
        df_results['pred_binary_'] = np.where((df_results.pred_ > i) , 1, 0)
        acc = accuracy_score(df_results['target_'], df_results['pred_binary_'])
        if acc > acc_best:
            df = df_results
            best_i = i
            acc_best = acc
            
    return acc_best, best_i, df


def best_treshold(df_results, t= []):
    acc_best = -999
    best_i = -999

    if t == []:
        for i in np.arange(0, 1, 0.1):
            df_results['pred_binary_'] = np.where((df_results.pred_ > i) , 1, 0)
            acc = accuracy_score(df_results['target_'], df_results['pred_binary_'])
            if acc > acc_best:
                df = df_results
                best_i = i
                acc_best = acc
    else:
        best_i = t[0]
        df_results['pred_binary'] = np.where((df_results.pred > t[0]) , 1, 0)
        df = df_results
    return acc_best, best_i, df

def pkl(id_, n_layers, n_heads, model_num, d_model):
    return '{}_d{}_layer{}_multihead{}_MODEL{}.pkl'.format(id_, d_model, n_layers, n_heads, model_num)


def make_data(data):
    pep_inputs, hla_inputs, labels = [], [], []
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
        labels.append(label)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs), torch.LongTensor(labels)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels

    def __len__(self):
        return self.pep_inputs.shape[0]

    def __getitem__(self, idx):
        return self.pep_inputs[idx], self.hla_inputs[idx], self.labels[idx]

In [8]:

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model)
        #self.pos_emb = PositionalEncoding(d_model)
        #self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        
        #enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        
        return enc_outputs, 0


class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        #self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        #print("\n\nAfter concat:", enc_outputs, "\nsize", np.shape(enc_outputs))
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        #dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = enc_outputs.view(enc_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        #print("len(dec_outputs), len(dec_outputs[0])", len(dec_outputs), len(dec_outputs[0]))
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), 0, 0, 0

In [9]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)



def performances_QM_(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('acc={:.4f}'.format(accuracy))
    
    return (fn, tp, accuracy)



def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])


# In[26]:


f_mean = lambda l: sum(l)/len(l)


# In[28]:


def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd


In [10]:
def train_step(model, train_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    time_train_ep = 0
    model.train()
    y_true_train_list, y_prob_train_list = [], []
    loss_train_list, dec_attns_train_list = [], []
    for train_pep_inputs, train_hla_inputs, train_labels in train_loader:
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        train_outputs: [batch_size, 2]
        '''
        train_pep_inputs, train_hla_inputs, train_labels = train_pep_inputs.to(device), train_hla_inputs.to(device), train_labels.to(device)

        t1 = time.time()
        train_outputs, _, _, train_dec_self_attns = model(train_pep_inputs, train_hla_inputs)
        train_loss = criterion(train_outputs, train_labels)
        time_train_ep += time.time() - t1

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        y_true_train = train_labels.cpu().numpy()
        y_prob_train = nn.Softmax(dim = 1)(train_outputs)[:, 1].cpu().detach().numpy()
        
        y_true_train_list.extend(y_true_train)
        y_prob_train_list.extend(y_prob_train)
        loss_train_list.append(train_loss)
#         dec_attns_train_list.append(train_dec_self_attns)
        
    y_pred_train_list = transfer(y_prob_train_list, threshold)
    ys_train = (y_true_train_list, y_pred_train_list, y_prob_train_list)
    
    print('Fold-{}****Train (Ep avg): Epoch-{}/{} | Loss = {:.4f} | Time = {:.4f} sec'.format(fold, epoch, epochs, f_mean(loss_train_list), time_train_ep))
    metrics_train = performances(y_true_train_list, y_pred_train_list, y_prob_train_list, print_ = True)
    
    return ys_train, loss_train_list, metrics_train, time_train_ep#, dec_attns_train_list


def eval_step(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print('Fold-{} ****Test  Epoch-{}/{}: Loss = {:.6f}'.format(fold, epoch, epochs, f_mean(loss_val_list)))
        
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

In [11]:
pep_max_len = 14 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load( data_dir + 'Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)

# Transformer Parameters
d_model = 20  # Embedding Size
d_k = d_v = 20  # dimension of K(=Q), V

d_ff = 512 # FeedForward dimension
n_layers = 1  # number of Encoder of Decoder Layer
n_heads = 3

batch_size = 1024
epochs = 25
threshold = 0.5

In [12]:
cv = 5
count = 0
index_order = []

for outer in range(cv):
        #print(f"\nk_outer: {outer}")
        #print("test: " + str(outer))
        test_idx = outer
        for inner in range(cv):
            #print(f"\t\nk_inner: {inner}")
            if inner!=outer:
                #print("\tval: " + str(inner))
                val_idx = inner
                train_index = list()
                for t in range(5):
                    if t!=inner and t!=outer:
                        train_index.append(t)
                #print("\ttrain:", str(train_index))
                count += 1  
                index_order.append([test_idx, val_idx, train_index])
                
index_order_nested = index_order


def data_with_loader(index_order_idx, type_ = 'train', fold = None,  batch_size = 1024):
    df_list = []
    for i in range(5):
        df_list.append(pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0)[0:per]) 
        
    if type_ != 'train' and type_ != 'val' and type_ != 'test':
        data = pd.read_csv(data_dir + '{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.concat([ df_list[index_order_nested[index_order_idx][2][0]], df_list[index_order_nested[index_order_idx][2][1]], df_list[index_order_nested[index_order_idx][2][2]] ]).reset_index().drop(["index"], axis=1)
        print("TRAIN ID:", index_order_nested[index_order_idx][2], len(data))
    elif type_ == 'test':
        data = df_list[index_order_nested[index_order_idx][0]]
        print("TEST ID:", index_order_nested[index_order_idx][0], len(data))
    elif type_ == 'val':
        data = df_list[index_order_nested[index_order_idx][1]]
        print("VAL ID:", index_order_nested[index_order_idx][1], len(data))
        
    pep_inputs, hla_inputs, labels = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    print('Fold-{} Label info: Data {} = {} '.format(fold, type_, Counter(data.label)))
    
    del df_list, data
    gc.collect()
    return pep_inputs, hla_inputs, labels, loader

index_order = 0

In [14]:
ys_train_fold_dict, ys_val_fold_dict = {}, {}
train_fold_metrics_list, val_fold_metrics_list, test_fold_metrics_list = [], [], []
#independent_fold_metrics_list, external_fold_metrics_list, ys_independent_fold_dict, ys_external_fold_dict = [], [], {}, {}
attns_train_fold_dict, attns_val_fold_dict = {}, {}
loss_train_fold_dict, loss_val_fold_dict, = {}, {}
metrics_all_val = [] 
metrics_all_train = [] 

time_train_c = time.time()

for fold in tqdm(range(20)): #####!!!!!!!

        print('=====Inner Fold-{}====='.format(fold))
        print('-----Generate data loader-----')
        index_order_idx = fold
        train_pep_inputs, train_hla_inputs, train_labels, train_loader = data_with_loader(index_order_idx, type_ = 'train', fold = fold,  batch_size = batch_size)
        val_pep_inputs, val_hla_inputs, val_labels, val_loader = data_with_loader(index_order_idx, type_ = 'val', fold = fold,  batch_size = batch_size)

        print('TEST ID', index_order_nested[index_order_idx][0])

        print('-----Compile model-----')
        model = Transformer().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 1e-3) #, momentum = 0.99)

        print('-----Train-----')
        path_saver = model_folder + pkl(id_, n_layers, n_heads, fold, d_model)

        print('path_saver: ', path_saver)

        metric_best, ep_best = 0, -1
        time_train = 0

        #if not os.path.exists(path_saver):
        for epoch in range(1, epochs + 1):
            time_train_f = time.time()
            ys_train, loss_train_list, metrics_train, time_train_ep = train_step(model, train_loader, fold, epoch, epochs, use_cuda) # , dec_attns_train
            ys_val, loss_val_list, metrics_val, _prob, _binary = eval_step(model, val_loader, fold, epoch, epochs, use_cuda) #, dec_attns_val

            train_fold_metrics_list.append(metrics_train)
            val_fold_metrics_list.append(metrics_val)

            metrics_ep_avg = sum(metrics_val[:4])/4
            if metrics_ep_avg > metric_best: 
                metric_best, ep_best = metrics_ep_avg, epoch
                torch.save(model.eval().state_dict(), path_saver)
            print("Fold time: {:6.2f} mins.".format(   round( (time.time()-time_train_f)/60, 2)   ))

        print("best_epoch:", ep_best)
        print('-----Optimization Finished!-----')
print("Total training time: {:6.2f} mins.".format(   round( (time.time()-time_train_c)/60, 2)   ))

  0%|                                                                                                             | 0/20 [00:00<?, ?it/s]

=====Inner Fold-0=====
-----Generate data loader-----
TRAIN ID: [2, 3, 4] 451500
Fold-0 Label info: Data train = Counter({1: 226074, 0: 225426}) 
VAL ID: 1 150500
Fold-0 Label info: Data val = Counter({0: 75567, 1: 74933}) 
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_25p_onlyEmbedding_NOPOS_d20_layer1_multihead3_MODEL0.pkl
Fold-0****Train (Ep avg): Epoch-1/25 | Loss = 0.2881 | Time = 0.3285 sec
tn = 197208, fp = 28218, fn = 26931, tp = 199143
y_pred: 0 = 224139 | 1 = 227361
y_true: 0 = 225426 | 1 = 226074
auc=0.9497|sensitivity=0.8809|specificity=0.8748|acc=0.8779|mcc=0.7557
precision=0.8759|recall=0.8809|f1=0.8784|aupr=0.9483
Fold-0 ****Test  Epoch-1/25: Loss = 0.251830
tn = 69476, fp = 6091, fn = 9311, tp = 65622
y_pred: 0 = 78787 | 1 = 71713
y_true: 0 = 75567 | 1 = 74933
auc=0.9627|sensitivity=0.8757|specificity=0.9194|acc=0.8977|mcc=0.7960
precision=0.9151|recall=0.8757|f1=0.8950|aupr=0.9607
Fo

Fold-0****Train (Ep avg): Epoch-14/25 | Loss = 0.1561 | Time = 0.3531 sec
tn = 211954, fp = 13472, fn = 13441, tp = 212633
y_pred: 0 = 225395 | 1 = 226105
y_true: 0 = 225426 | 1 = 226074
auc=0.9848|sensitivity=0.9405|specificity=0.9402|acc=0.9404|mcc=0.8808
precision=0.9404|recall=0.9405|f1=0.9405|aupr=0.9845
Fold-0 ****Test  Epoch-14/25: Loss = 0.205660
tn = 70338, fp = 5229, fn = 6355, tp = 68578
y_pred: 0 = 76693 | 1 = 73807
y_true: 0 = 75567 | 1 = 74933
auc=0.9755|sensitivity=0.9152|specificity=0.9308|acc=0.9230|mcc=0.8461
precision=0.9292|recall=0.9152|f1=0.9221|aupr=0.9750
Fold time:   0.20 mins.
Fold-0****Train (Ep avg): Epoch-15/25 | Loss = 0.1531 | Time = 0.3400 sec
tn = 212191, fp = 13235, fn = 13136, tp = 212938
y_pred: 0 = 225327 | 1 = 226173
y_true: 0 = 225426 | 1 = 226074
auc=0.9854|sensitivity=0.9419|specificity=0.9413|acc=0.9416|mcc=0.8832
precision=0.9415|recall=0.9419|f1=0.9417|aupr=0.9851
Fold-0 ****Test  Epoch-15/25: Loss = 0.206613
tn = 70238, fp = 5329, fn = 6265,

  5%|████▉                                                                                             | 1/20 [05:27<1:43:37, 327.25s/it]

tn = 70567, fp = 5000, fn = 7353, tp = 67580
y_pred: 0 = 77920 | 1 = 72580
y_true: 0 = 75567 | 1 = 74933
auc=0.9730|sensitivity=0.9019|specificity=0.9338|acc=0.9179|mcc=0.8362
precision=0.9311|recall=0.9019|f1=0.9163|aupr=0.9722
Fold time:   0.22 mins.
best_epoch: 14
-----Optimization Finished!-----
=====Inner Fold-1=====
-----Generate data loader-----
TRAIN ID: [1, 3, 4] 451500
Fold-1 Label info: Data train = Counter({1: 225912, 0: 225588}) 
VAL ID: 2 150500
Fold-1 Label info: Data val = Counter({0: 75405, 1: 75095}) 
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_25p_onlyEmbedding_NOPOS_d20_layer1_multihead3_MODEL1.pkl
Fold-1****Train (Ep avg): Epoch-1/25 | Loss = 0.2850 | Time = 0.3953 sec
tn = 197241, fp = 28347, fn = 26453, tp = 199459
y_pred: 0 = 223694 | 1 = 227806
y_true: 0 = 225588 | 1 = 225912
auc=0.9508|sensitivity=0.8829|specificity=0.8743|acc=0.8786|mcc=0.7573
precision=0.8756|recall=0.88

...

 95%|████████████████████████████████████████████████████████████████████████████████████████████▏    | 19/20 [1:43:25<05:31, 331.12s/it]

tn = 70395, fp = 5010, fn = 7458, tp = 67637
y_pred: 0 = 77853 | 1 = 72647
y_true: 0 = 75405 | 1 = 75095
auc=0.9724|sensitivity=0.9007|specificity=0.9336|acc=0.9172|mcc=0.8347
precision=0.9310|recall=0.9007|f1=0.9156|aupr=0.9715
Fold time:   0.22 mins.
best_epoch: 15
-----Optimization Finished!-----
=====Inner Fold-19=====
-----Generate data loader-----
TRAIN ID: [0, 1, 2] 451500
Fold-19 Label info: Data train = Counter({0: 226198, 1: 225302}) 
VAL ID: 3 150500
Fold-19 Label info: Data val = Counter({0: 75303, 1: 75197}) 
TEST ID 4
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_25p_onlyEmbedding_NOPOS_d20_layer1_multihead3_MODEL19.pkl
Fold-19****Train (Ep avg): Epoch-1/25 | Loss = 0.2864 | Time = 0.3450 sec
tn = 197964, fp = 28234, fn = 26892, tp = 198410
y_pred: 0 = 224856 | 1 = 226644
y_true: 0 = 226198 | 1 = 225302
auc=0.9503|sensitivity=0.8806|specificity=0.8752|acc=0.8779|mcc=0.7558
precision=0.8754|recall

Fold-19 ****Test  Epoch-13/25: Loss = 0.207532
tn = 70212, fp = 5091, fn = 6763, tp = 68434
y_pred: 0 = 76975 | 1 = 73525
y_true: 0 = 75303 | 1 = 75197
auc=0.9752|sensitivity=0.9101|specificity=0.9324|acc=0.9212|mcc=0.8427
precision=0.9308|recall=0.9101|f1=0.9203|aupr=0.9746
Fold time:   0.21 mins.
Fold-19****Train (Ep avg): Epoch-14/25 | Loss = 0.1507 | Time = 0.3980 sec
tn = 213231, fp = 12967, fn = 12926, tp = 212376
y_pred: 0 = 226157 | 1 = 225343
y_true: 0 = 226198 | 1 = 225302
auc=0.9858|sensitivity=0.9426|specificity=0.9427|acc=0.9427|mcc=0.8853
precision=0.9425|recall=0.9426|f1=0.9425|aupr=0.9856
Fold-19 ****Test  Epoch-14/25: Loss = 0.211458
tn = 70536, fp = 4767, fn = 7323, tp = 67874
y_pred: 0 = 77859 | 1 = 72641
y_true: 0 = 75303 | 1 = 75197
auc=0.9749|sensitivity=0.9026|specificity=0.9367|acc=0.9197|mcc=0.8398
precision=0.9344|recall=0.9026|f1=0.9182|aupr=0.9741
Fold time:   0.22 mins.
Fold-19****Train (Ep avg): Epoch-15/25 | Loss = 0.1478 | Time = 0.3485 sec
tn = 213495, 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [1:48:52<00:00, 326.65s/it]

y_pred: 0 = 77346 | 1 = 73154
y_true: 0 = 75303 | 1 = 75197
auc=0.9722|sensitivity=0.9025|specificity=0.9298|acc=0.9161|mcc=0.8326
precision=0.9277|recall=0.9025|f1=0.9149|aupr=0.9717
Fold time:   0.24 mins.
best_epoch: 12
-----Optimization Finished!-----
Total training time: 108.88 mins.


In [14]:
def data_with_loader_unique(index_order_idx, type_ = 'train', fold = None,  batch_size = 1024):
    df_list = []
    for i in range(5):
        df = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0).drop_duplicates()
        df_list.append(df)
        
    if type_ != 'train' and type_ != 'val' and type_ != 'test':
        data = pd.read_csv(data_dir + '{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.concat([ df_list[index_order_nested[index_order_idx][2][0]], df_list[index_order_nested[index_order_idx][2][1]], df_list[index_order_nested[index_order_idx][2][2]] ]).reset_index().drop(["index"], axis=1)
        print("TRAIN ID:", index_order_nested[index_order_idx][2], len(data))
    elif type_ == 'test':
        data = df_list[index_order_nested[index_order_idx][0]]
        print("TEST ID:", index_order_nested[index_order_idx][0], len(data))
    elif type_ == 'val':
        data = df_list[index_order_nested[index_order_idx][1]]
        print("VAL ID:", index_order_nested[index_order_idx][1], len(data))
        
    pep_inputs, hla_inputs, labels = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    print('Label info: Data {} = {} '.format(fold, type_, Counter(data.label)))
    
    del df_list
    gc.collect()
    return data, pep_inputs, hla_inputs, labels, loader

def eval_step_test(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)
            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print_bool = False
        if epoch == 1 or epoch == 25 or epoch == 50:
            print_bool = True
            print('\nFold-{} **** Test  Epoch-{}/{}: Loss = {:.6f} | '.format(fold, epoch, epochs, f_mean(loss_val_list)), end='')
            
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list, print_ = print_bool)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

def binary_roc_auc_score(y_true, y_score, sample_weight=None, max_fpr=None):
    """Binary roc auc score."""
    if len(np.unique(y_true)) != 2:
        raise ValueError(
            "Only one class present in y_true. ROC AUC score "
            "is not defined in that case.")

    fpr, tpr, _ = roc_curve(y_true, y_score, sample_weight=sample_weight)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected max_fpr in range (0, 1], got: %r" % max_fpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    return partial_auc/0.1

In [ ]:
id_nested = id_

ep_best = 0
criterion = nn.CrossEntropyLoss()

test_fold_metrics_list_nested = []

fold_avg_4 = True
type_ = 'test'

df_list = []
time_testing = time.time()

if fold_avg_4:
    
    fold_avg = [list(range(0,4)), list(range(4,8)), list(range(8,12)), list(range(12,16)), list(range(16,20))] 
    pred_all = []
    target_all = []

    for fold in fold_avg:

            pred_fold = []
            target_fold = []
            file_num = fold[0]
            print('\n=====Models-{}====='.format(fold))
            
            for f in fold:
                index_order_idx = f
                test_data, test_pep_inputs, test_hla_inputs, test_labels, test_loader = data_with_loader_unique(index_order_idx, type_ = type_, fold = file_num,  batch_size = batch_size)
                model = Transformer().to(device)
                path_saver = model_folder + pkl(id_nested, n_layers, n_heads, f, d_model)
                print(path_saver)
                #
                model.load_state_dict(torch.load(path_saver))
                model_test = model.eval()

                ys_res_test, loss_res_test_list, metrics_res_test, y_prob, y_pred_bin = eval_step_test(model_test, test_loader, fold, ep_best, epochs, use_cuda) # , test_res_attns
                pred_fold.append(y_prob)
                target_fold = [x.item() for x in test_labels]
                
                ## add auc01 and ppv
                df_temp = pd.DataFrame(list(zip(target_fold, y_prob)), columns= ['target_', 'pred_'])
                num_pos = len(df_temp[df_temp['target_']==1])
                df_temp_ppv = df_temp.sort_values(by=['pred_'], ascending=False)[0:num_pos]
                num_true_pos = len(df_temp_ppv[df_temp_ppv['target_']==1])
                ppv = num_true_pos/num_pos
                auc_01 = binary_roc_auc_score(df_temp['target_'].tolist(), df_temp['pred_'], max_fpr=0.1)
                auc01_ppv = (auc_01, ppv)
                metrics_final = (*auc01_ppv, *metrics_res_test)
                
                test_fold_metrics_list_nested.append(metrics_final)
                
                print("\t\t\t\t roc in:", roc_auc_score(target_fold, y_prob))

            # get avg of pred_fold and extend pred_all
            arrays = [np.array(x) for x in pred_fold]
            pred_fold_avg = [np.mean(k) for k in zip(*arrays)]
            pred_all.extend(pred_fold_avg)
            target_all.extend(target_fold)
            print("\t\t roc avg accmmltd:", roc_auc_score(target_all, pred_all))  
            
            test_data['target_'] = target_fold
            test_data['pred_'] = pred_fold_avg
            df_list.append(test_data)


    df_pd_nested_test_unique = pd.concat(df_list)
    del df_list
    gc.collect()
    
print("Time testing:", round((time.time()-time_testing)/60, 2), "mins. \n\n")

In [ ]:
df_results = pd.DataFrame(list(zip(target_all, pred_all)), columns= ['target', 'pred'])
print("\nSTATS:")
auc_ = roc_auc_score(df_results['target'], df_results['pred'])
print("AUC:", auc_)


num_pos = len(df_results[df_results['target']==1])
df_temp_ppv = df_results.sort_values(by=['pred'], ascending=False)[0:num_pos]
num_true_pos = len(df_temp_ppv[df_temp_ppv['target']==1])
ppv = num_true_pos/num_pos
auc_01 = binary_roc_auc_score(df_results['target'].tolist(), df_results['pred'].tolist(), max_fpr=0.1)
print("AUC FPR 0.1:", auc_01)
print("PPV:", ppv)


print("\nPerformance adv:")

acc_best, i, df_results = best_treshold(df_results, [0.5])
print("Used/Best treshold:", i, "\n")
print("AUC pred treshold:", roc_auc_score(df_results['target'], df_results['pred_binary']))
roc_auc, acc, mcc, f1, sensitivity, specificity, precision, recall, aupr = performances(df_results['target'], df_results['pred_binary'], df_results['pred'], print_ = True)

In [ ]:
file = open(df_name + 'test_avg4.csv', "w")

file.write(f"#INFO:\n")
file.write(f"#Test type: Every for of 20 model is tested on a partition, avarage is taken.\n")
file.write(f"#id: {id_}\n")
file.write(f"#d_model: {d_model}\n")
file.write(f"#d_k, d_v: {d_k}, {d_v}\n")
file.write(f"#n_heads: {n_heads}\n")
file.write(f"#n_layers: {n_layers}\n")
file.write(f"#ff_layers: {d_ff}\n")
file.write(f"#epochs: {epochs}\n")
file.write(f"#treshold: {threshold}\n")

file.write(''.join("peptide,length,HLA,label,HLA_sequence,target_,pred_\n"))

for index, row in df_pd_nested_test_unique.iterrows():
    txt = row['peptide'] + "," + str(row['length']) + "," + row['HLA'] + "," + str(row['label']) + "," + row['HLA_sequence'] + "," + str(row['target_']) + "," + str(row['pred_']) + "\n"
    file.write(''.join(txt))

file.close()

print("DONE")

In [ ]:
attr = ['id', 'data', 'data_size', 'data_info', 'encoding', 'results_df', 'pep_max_len', 'hla_max_len', 'tgt_len', 'd_model', 'd_k', 'd_v', 'd_ff', 'n_layers', 'n_heads', 'batch_size', 'epochs', 'treshold', 'test_dup', 'test_type', 'auc', 'auc01', 'ppv', 'acc', 'mcc', 'sensitivity', 'specificity', 'precision', 'recall', 'f1', 'aupr']

test_dup = 'tested on unique'

global_info = [id_, data_ , size, per, encoding_ , df_name + 'test_avg4.csv', pep_max_len, hla_max_len, tgt_len, d_model, d_k, d_v, d_ff, n_layers, n_heads, batch_size, epochs, threshold, test_dup, 'average of each 4 - then concat', auc_, auc_01, ppv, acc, mcc, sensitivity, specificity, precision, recall, f1, aupr]

for i,t in zip(global_info, attr):
    print(t, ":", i)
    
with open(result_folder + 'results_global', 'a') as file:
    file.write('\n'+','.join([str(x) for x in global_info]))
file.close()